In [1]:
from flask import Flask, jsonify 
from flask_restful import Resource, Api , reqparse
import pandas as pd 

In [2]:
app=Flask(__name__)
api=Api(app)

In [3]:
data_arg=reqparse.RequestParser()
data_arg.add_argument("ID" , type=int ,help="Enter ID", location='form')
data_arg.add_argument("Name" , type=str ,help="Enter Name", location='form')
data_arg.add_argument("Language" , type=str ,help="Enter Language", location='form')
data_arg.add_argument("Age" , type=int ,help="Enter Age", location='form')

In [4]:
class read_Delete(Resource):
    def __init__(self):
        # read csv file
        self.data = pd.read_csv('dataset_test.csv')
    # GET request on the url will hit this function
    def get(self,ID):
        # find data from csv based on user input
        data_fount=self.data.loc[self.data['ID'] == ID].to_json(orient="records")
        # return data found in csv
        return jsonify({'message': data_fount})
    # Delete request on the url will hit this function
    def delete(self,ID):
        if ((self.data['ID'] == ID).any()):
            # Id it present delete data from csv
            self.data = self.data.drop(self.data["ID"].loc[self.data["ID"] == ID].index)
            self.data.to_csv("dataset_test.csv", index=False)
            return jsonify({"message": 'Deleted successfully'})
        else:
            return jsonify({"message": 'Not Present'})

In [5]:
class Create_Update(Resource):
    def __init__(self):
        # read data from csv
        self.data = pd.read_csv('dataset_test.csv')

    # POST request on the url will hit this function
    def post(self):
        # data parser to parse data from url
        args = data_arg.parse_args()
        # if ID is already present
        if((self.data['ID']==args.ID).any()):
            return jsonify({"message": 'ID already exist'})
        else:
            # Save data to csv
            self.data= self.data.append(args, ignore_index=True)
            self.data.to_csv("dataset_test.csv", index=False)
            return jsonify({"message": 'Done'})

    # PUT request on the url will hit this function
    def put(self):
        args = data_arg.parse_args()
        if ((self.data['ID'] == args.ID).any()):
            # if ID already present Update it
            self.data=self.data.drop(self.data["ID"].loc[self.data["ID"] == args.ID].index)
            self.data = self.data.append(args, ignore_index=True)
            self.data.to_csv("dataset_test.csv", index=False)
            return jsonify({"message": 'Updated successfully'})
        else:
            # If ID not present Save that data to csv
            self.data = self.data.append(args, ignore_index=True)
            self.data.to_csv("dataset_test.csv", index=False)
            return jsonify({"message": 'successfully Created'})

In [6]:
api.add_resource(read_Delete, '/<int:ID>')
api.add_resource(Create_Update,'/')

In [ ]:
if __name__ == '__main__':
    #app.run(debug=True)
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Nov/2022 19:54:50] "GET / HTTP/1.1" 405 -
127.0.0.1 - - [17/Nov/2022 19:54:50] "GET /favicon.ico HTTP/1.1" 404 -
